In [1]:
from db import Database
from modules import VLM
db = Database()
db.connect()

/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scur0279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/scur0279/MMA/.venv/lib64/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [2]:
users = db.fetch_all_users()
display(users)

,id,username,password
0,1,first_test_user,password123
1,2,second_test_user,password456


### Graph 1

In [3]:
# is_selected can be true, false, or None
images_df = db.fetch_images_by_user(1, include_depth=True, is_selected=True)
images_df.head()
# display(images_df)

,id,input_prompt_id,output_prompt_id,user_id,chat_id,prompt_guidance,image_guidance,path,selected,depth
0,cbd74fbd-521e-4838-a70b-5e7e955d3713,4fc8c199-c6df-4578-b8b2-302722da69a7,4fc8c199-c6df-4578-b8b2-302722da69a7,1,1,6.0,0.4,images/cbd74fbd-521e-4838-a70b-5e7e955d3713.png,1,1
1,581539f7-ebc9-4325-b2d8-b13bff20d439,78e321aa-1c3e-4e37-8be6-a8cadb5b1312,78e321aa-1c3e-4e37-8be6-a8cadb5b1312,1,1,6.0,0.4,images/581539f7-ebc9-4325-b2d8-b13bff20d439.png,1,2
2,6fa94a73-f7ed-4aad-b2ed-16facabe0361,9ae88630-bb20-417e-a782-0867c5a6ce4a,9ae88630-bb20-417e-a782-0867c5a6ce4a,1,1,6.0,0.4,images/6fa94a73-f7ed-4aad-b2ed-16facabe0361.png,1,3
3,e1a3fe54-f85d-4e86-a86f-9d076032f2e2,1d4ef934-bc6e-410b-bc7b-aed64c1b65e0,1d4ef934-bc6e-410b-bc7b-aed64c1b65e0,1,2,7.0,0.5,images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png,1,3
4,551014ae-ed6d-44b8-9c87-70e2b89a0109,e94e184b-0a19-4926-b72d-8f3aaa026c87,e94e184b-0a19-4926-b72d-8f3aaa026c87,1,2,7.0,0.5,images/551014ae-ed6d-44b8-9c87-70e2b89a0109.png,1,1


In [4]:
depth_stats = images_df.groupby('depth').agg({
    'prompt_guidance': 'mean',
    'image_guidance': 'mean'
}).round(3)  # Round to 3 decimal places for cleaner output

print(depth_stats)

depths = depth_stats.index.tolist()
mean_prompt_guidance = depth_stats['prompt_guidance'].tolist()
mean_image_guidance = depth_stats['image_guidance'].tolist()

print("depths:", depths)
print("Mean prompt guidance values:", mean_prompt_guidance)
print("Mean image guidance values:", mean_image_guidance)

       prompt_guidance  image_guidance
depth                                 
1                  6.5            0.45
2                  6.5            0.45
3                  6.5            0.45
depths: [1, 2, 3]
Mean prompt guidance values: [6.5, 6.5, 6.5]
Mean image guidance values: [0.45, 0.45, 0.45]


### Graph 2

In [5]:
chat_ids = sorted(set(images_df['chat_id'].tolist()))
print(chat_ids)

for chat_id in chat_ids:
    print(f"Chat ID: {chat_id}")
    chat_images = images_df[images_df['chat_id'] == chat_id]
    display(chat_images)

    #TODO calculate bert lpips scores based on these sequential (depth) images

[1, 2]
Chat ID: 1


,id,input_prompt_id,output_prompt_id,user_id,chat_id,prompt_guidance,image_guidance,path,selected,depth
0,cbd74fbd-521e-4838-a70b-5e7e955d3713,4fc8c199-c6df-4578-b8b2-302722da69a7,4fc8c199-c6df-4578-b8b2-302722da69a7,1,1,6.0,0.4,images/cbd74fbd-521e-4838-a70b-5e7e955d3713.png,1,1
1,581539f7-ebc9-4325-b2d8-b13bff20d439,78e321aa-1c3e-4e37-8be6-a8cadb5b1312,78e321aa-1c3e-4e37-8be6-a8cadb5b1312,1,1,6.0,0.4,images/581539f7-ebc9-4325-b2d8-b13bff20d439.png,1,2
2,6fa94a73-f7ed-4aad-b2ed-16facabe0361,9ae88630-bb20-417e-a782-0867c5a6ce4a,9ae88630-bb20-417e-a782-0867c5a6ce4a,1,1,6.0,0.4,images/6fa94a73-f7ed-4aad-b2ed-16facabe0361.png,1,3


Chat ID: 2


,id,input_prompt_id,output_prompt_id,user_id,chat_id,prompt_guidance,image_guidance,path,selected,depth
3,e1a3fe54-f85d-4e86-a86f-9d076032f2e2,1d4ef934-bc6e-410b-bc7b-aed64c1b65e0,1d4ef934-bc6e-410b-bc7b-aed64c1b65e0,1,2,7.0,0.5,images/e1a3fe54-f85d-4e86-a86f-9d076032f2e2.png,1,3
4,551014ae-ed6d-44b8-9c87-70e2b89a0109,e94e184b-0a19-4926-b72d-8f3aaa026c87,e94e184b-0a19-4926-b72d-8f3aaa026c87,1,2,7.0,0.5,images/551014ae-ed6d-44b8-9c87-70e2b89a0109.png,1,1
5,7bdb419c-ac2b-4980-a5b8-03debc54a264,fe69e976-8796-4643-9d7f-8efe20a4244d,fe69e976-8796-4643-9d7f-8efe20a4244d,1,2,7.0,0.5,images/7bdb419c-ac2b-4980-a5b8-03debc54a264.png,1,2


### Graph 3

In [6]:
prompts_df = db.fetch_prompts_by_user(1)
display(prompts_df)
# prompts_df.head()

,id,user_id,chat_id,prompt,depth,used_suggestion,modified_suggestion,suggestion_used,is_enhanced,enhanced_prompt,image_in_id,images_out
0,4fc8c199-c6df-4578-b8b2-302722da69a7,1,1,Generate a bottle of perfume,1,0,0,None,0,None,None,None
1,78e321aa-1c3e-4e37-8be6-a8cadb5b1312,1,1,Generate a bottle of perfume in a desert,2,0,0,None,0,None,None,None
2,9ae88630-bb20-417e-a782-0867c5a6ce4a,1,1,Add sunset lighting to the perfume bottle scene,3,1,0,Try sunset colors,1,A perfume bottle in a desert with glowing suns...,None,None
3,e94e184b-0a19-4926-b72d-8f3aaa026c87,1,2,Design a futuristic drone,1,0,0,None,0,None,None,None
4,fe69e976-8796-4643-9d7f-8efe20a4244d,1,2,Add neon lighting to the drone,2,0,0,None,0,None,None,None
5,1d4ef934-bc6e-410b-bc7b-aed64c1b65e0,1,2,Place the drone in a cyberpunk city,3,1,1,Try placing it in a city,0,None,None,None


In [7]:
stat1 = len(prompts_df[(prompts_df['used_suggestion'] == False) & (prompts_df['is_enhanced'] == False)]) / len(prompts_df) * 100
stat2 = len(prompts_df[(prompts_df['used_suggestion'] == True) & (prompts_df['is_enhanced'] == False)]) / len(prompts_df) * 100
stat3 = len(prompts_df[(prompts_df['is_enhanced'] == True) & (prompts_df['used_suggestion'] == False)]) / len(prompts_df) * 100
stat4 = len(prompts_df[(prompts_df['used_suggestion'] == True) & (prompts_df['is_enhanced'] == True)]) / len(prompts_df) * 100

print("Prompt Statistics:")
print(f"1) Both used_suggestion and is_enhanced are False: {stat1}")
print(f"2) used_suggestion is True and is_enhanced is False: {stat2}")
print(f"3) is_enhanced is True and used_suggestion is False: {stat3}")
print(f"4) Both used_suggestion and is_enhanced are True: {stat4}")

values = [stat1, stat2, stat3, stat4]

print(values)

Prompt Statistics:
1) Both used_suggestion and is_enhanced are False: 66.66666666666666
2) used_suggestion is True and is_enhanced is False: 16.666666666666664
3) is_enhanced is True and used_suggestion is False: 0.0
4) Both used_suggestion and is_enhanced are True: 16.666666666666664
[66.66666666666666, 16.666666666666664, 0.0, 16.666666666666664]


### Graph 4

In [8]:
all_prompts = prompts_df['prompt'].tolist()
print(all_prompts)

['Generate a bottle of perfume', 'Generate a bottle of perfume in a desert', 'Add sunset lighting to the perfume bottle scene', 'Design a futuristic drone', 'Add neon lighting to the drone', 'Place the drone in a cyberpunk city']


In [9]:
#TODO I am not entirely sure how you want to make the wordclouds / what keywords you want.
# maybe you can use:

# might have to use snellius (in a .py file) for this as it could be quite slow
# vlm = VLM()
# for prompt in all_prompts:ii
#     keywords = vlm.extract_keywords(prompt)